In [4]:
import numpy as np

In [35]:
def els_price(stock_price, security_price, maturity, vol, interest, dividend_rate, barrier_level, K, profit):
    s0 = stock_price
    T = maturity
    sigma = vol
    d = dividend_rate
    b = barrier_level
    payoff = [i*profit/2 for i in range(1,7)]
    r = interest
    exercise_date = [252*i/2 for i in range(1,7)]
    dt = 1/252
    nth = len(exercise_date) - 1

    s_max = s0 * 5
    #브라우니안 모션을 가정하면, ELS 동안 가장 높게 올라갈 수 있는 건 1차 조기 상환일까지 주가가 계속 올라가기만 했을 때의 주가이다.
    #Normal 분포이기 때문에 올라갈 수 있는 주가가 무한하다.. 브라우니안으로 잡는건 문제가 있다.
    s_min = 0
    Jsize = 1000
    s_arr = np.linspace(s_min,s_max,Jsize+1)
    h = (s_max-s_min)/Jsize
    
    t_min, t_max = 0, T*252
    Nsize = t_max - t_min
    t_arr = np.arange(t_min, t_max + 1)
    k = dt
    
    u = np.empty((Nsize + 1, Jsize + 1)) # knock in 안 된 ELS Value 격자
    knockin_u = np.empty((Nsize + 1, Jsize + 1)) # knock in 된 ELS Value 격자
    
    #초기조건에 대한 값을 두 격자판에 입력
    #만기에서 조기상환조건을 만족하면 full dummy 그리고 barrier를 안쳤어도 full dummy, barrier를 쳤으면 s 그대로 나옴
    T_payoff_u = [1 + payoff[-1] if s >= s0 * K[-1] else 1 + payoff[-1] if s >= s0 * b else s for s in s_arr] # s가 아니라 s/s0아닌가 ?
    #만기에서 조기상환조건을 만족하면 full dummy인데, 조기상환조건 만족못하면 knock in 된거니까 가차없이 손실을 뱉어냄
    T_payoff_knockin_u = [1 + payoff[-1] if s >= s0 * K[-1] else s/s0 for s in s_arr]
    
    u[-1,:] = np.array(T_payoff_u)
    knockin_u[-1, :] = np.array(T_payoff_knockin_u)
    
    #앞으로 돌면서 격자판을 채워나감, 계수들을 먼저 채우고, ELS value 값을 채움
    #계수들
    #블로그에는 S_j를 돌리는데, 손으로 계산해봤을 때는, j가 그대로 들어감, S_j = S변화량 * j 이거 때문에.
    coef = np.array([1+sigma**2 * i**2 * dt + r * dt for i in range (1,Jsize)])
    coef_upper = np.array([-(1/2*sigma**2 * i**2 * dt + r/2*dt*i) for i in range(1,Jsize)])
    coef_under = np.array([-(1/2*sigma**2 * i**2 * dt - r/2*dt*i) for i in range(1,Jsize)])

    #경계조건을 고려한 계수, S_max에서는 감마중립
    coef[0] = 2 * coef_under[0] + coef[0]
    coef_upper[0] = - coef_under[0] + coef_upper[0]
    
    coef[-1] = coef[-1] + 2 * coef_upper[-1]
    coef_under[-1] = coef_under[-1] - coef_upper[-1]
    
    
    for i in range (Nsize + 1)[::-1]:
        #뒤에서 앞으로 가면서 한 줄씩(세로로) 채워나간다. 이때 J(주가)는 경계 빼고 나머지 부분을 채워나감
        known_u, known_ku = u[i, 1:Jsize], knockin_u[i, 1:Jsize]
        #채운부분을 통해 thomas 알고리즘으로 앞 시점 모르는 줄을 채워나감
        unknown_u, unknown_ku = thomas(coef_under, coef, coef_upper, known_u), thomas(coef_under, coef, coef_upper, known_ku)
        
        u[i-1,1:Jsize] = unknown_u
        knockin_u[i-1, 1:Jsize] = unknown_ku
        
        #n-1시점, boundary에 있는 값들을 채워준 값들로 만들어줌. 채운건 1~Jsize까지 였으니까 0와 -1은 비어있는 상태
        u[i - 1, 0] = 2 * u[i - 1, 1] - u[i - 1, 2]
        u[i - 1, -1] = 2 * u[i - 1, -2] - u[i - 1, -3]

        knockin_u[i - 1, 0] = 2 * knockin_u[i - 1, 1] - knockin_u[i - 1, 2]
        knockin_u[i - 1, -1] = 2 * knockin_u[i - 1, -2] - knockin_u[i - 1, -3]

        #knock in과 not knock in을 결합
        u[i - 1, :] = np.where(s_arr >= s0*b, u[i - 1, :], knockin_u[i - 1, :])
    
        #조기상환 시점이 되었을 때
        if i == exercise_date[nth]:
            u[i, :] = np.where(s_arr >= s0 * K[nth], 1 + payoff[nth], u[i, :])
            knockin_u[i, :] = np.where(s_arr >= s0 * K[nth], 1 + payoff[nth], knockin_u[i, :])
            nth-= 1

        ELS_value = np.interp(stock_price, s_arr, u[0, :])
        
        return ELS_value

In [7]:
def thomas(a, b, c, d):
    """ A is the tridiagnonal coefficient matrix and d is the RHS matrix"""
    """
    a is lower diagonal a2,a3,..,a_N, meaning
    b is diagonal b1,b2,b3,..,b_N meaning
    c is upper diagonal c1,c2,c3,.. c_{N-1} meaning
    """
    N = len(a)
    cp = np.zeros(N, dtype='float64')  # store tranformed c or c'
    dp = np.zeros(N, dtype='float64')  # store transformed d or d'
    X = np.zeros(N, dtype='float64')  # store unknown coefficients

    # Perform Forward Sweep
    # Equation 1 indexed as 0 in python
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    # Equation 2, ..., N (indexed 1 - N-1 in Python)
    for i in np.arange(1, (N), 1):
        dnum = b[i] - a[i] * cp[i - 1]
        cp[i] = c[i] / dnum
        dp[i] = (d[i] - a[i] * dp[i - 1]) / dnum

    # Perform Back Substitution
    X[(N - 1)] = dp[N - 1]  # Obtain last xn

    for i in np.arange((N - 2), -1, -1):  # use x[i+1] to obtain x[i]
        X[i] = (dp[i]) - (cp[i]) * (X[i + 1])

    return (X)

In [33]:
#parameter들
stock_price = 726000
sp = 10000
mat = 3
vol = 0.4217
interest = 0.034
div = 0.017
barrier = 0.4
K = [0.9, 0.85, 0.8, 0.8, 0.75, 0.75]
profit = 0.1136 #연 11.36%



In [36]:
ELS_price = els_price(stock_price, sp, mat, vol, interest, div, barrier, K, profit)
print(ELS_price)

0.0
